In [2]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import f1_score
from itertools import combinations
from tqdm.notebook import trange
from ast import literal_eval

# Internal Imports
import sys
sys.path.insert(0,'../')
from data_utils import add_deltas_and_time, add_lifetime, add_extreme

In [2]:
# Datafile
df = pd.read_csv("../NA850data_2000-2014.csv")
df = add_deltas_and_time(df)
df = add_extreme(df)

In [3]:
df

,AirTemp,QV,Omega,SeaLevPress,UWinds,VWinds,LH,Lat,Lon,PtIndex,...,dAirTemp,dQV,dOmega,dSeaLevPress,dUWinds,dVWinds,dLH,dLat,dLon,Extreme
Time,,,,,,,,,,,,,,,,,,,,,
2000-01-01 03:00:00,7.100183,7.474899,-0.000296,1021.084141,-11.230261,-3.172798,-10.217120,24.480576,-36.771637,1.0,...,0.039337,0.207899,-0.000937,-0.481484,0.664170,2.103366,-23.105201,-0.583883,-1.402740,1
2000-01-01 06:00:00,7.068231,7.335827,0.000323,1020.302969,-10.246526,-1.993664,0.511768,24.649042,-37.882874,2.0,...,-0.031952,-0.139073,0.000619,-0.781172,0.983735,1.179135,10.728888,0.168467,-1.111237,0
2000-01-01 09:00:00,7.328638,7.064822,0.000828,1020.585156,-9.889462,-1.943067,-16.420087,24.122194,-38.753021,3.0,...,0.260406,-0.271005,0.000505,0.282187,0.357064,0.050596,-16.931855,-0.526848,-0.870148,1
2000-01-01 12:00:00,7.598169,7.333828,0.000373,1022.448281,-10.461138,-1.204948,-6.642615,24.368916,-40.022919,4.0,...,0.269531,0.269006,-0.000455,1.863125,-0.571676,0.738119,9.777472,0.246721,-1.269897,0
2000-01-01 15:00:00,8.023950,7.629423,0.000186,1020.875625,-10.675546,-1.058556,-0.921751,24.508623,-41.214752,5.0,...,0.425781,0.295595,-0.000188,-1.572656,-0.214408,0.146392,5.720864,0.139708,-1.191833,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-13 21:00:00,13.976068,9.267782,0.000962,1013.787969,-5.725973,1.024751,8.022283,23.499479,-48.751160,51.0,...,-0.027283,0.573225,0.001772,1.036875,0.239570,-0.696071,-1.964289,-0.036551,-0.382538,0
2014-12-14 00:00:00,13.713007,8.729606,-0.000825,1014.931094,-5.089421,3.418641,5.292976,23.499500,-49.022491,52.0,...,-0.263062,-0.538176,-0.001787,1.143125,0.636552,2.393891,-2.729308,0.000021,-0.271332,0
2014-12-14 03:00:00,14.085962,9.510149,0.000713,1014.133594,-3.518239,3.075252,4.205419,23.383141,-49.258881,53.0,...,0.372955,0.780543,0.001538,-0.797500,1.571182,-0.343390,-1.087557,-0.116360,-0.236389,0


In [4]:
df = add_lifetime(df)

In [5]:
df

,AirTemp,QV,Omega,SeaLevPress,UWinds,VWinds,LH,Lat,Lon,PtIndex,...,dQV,dOmega,dSeaLevPress,dUWinds,dVWinds,dLH,dLat,dLon,Extreme,LifeTime
Time,,,,,,,,,,,,,,,,,,,,,
2000-01-01 03:00:00,7.100183,7.474899,-0.000296,1021.084141,-11.230261,-3.172798,-10.217120,24.480576,-36.771637,1.0,...,0.207899,-0.000937,-0.481484,0.664170,2.103366,-23.105201,-0.583883,-1.402740,1,0.0
2000-01-01 06:00:00,7.068231,7.335827,0.000323,1020.302969,-10.246526,-1.993664,0.511768,24.649042,-37.882874,2.0,...,-0.139073,0.000619,-0.781172,0.983735,1.179135,10.728888,0.168467,-1.111237,0,0.0
2000-01-01 09:00:00,7.328638,7.064822,0.000828,1020.585156,-9.889462,-1.943067,-16.420087,24.122194,-38.753021,3.0,...,-0.271005,0.000505,0.282187,0.357064,0.050596,-16.931855,-0.526848,-0.870148,1,0.0
2000-01-01 12:00:00,7.598169,7.333828,0.000373,1022.448281,-10.461138,-1.204948,-6.642615,24.368916,-40.022919,4.0,...,0.269006,-0.000455,1.863125,-0.571676,0.738119,9.777472,0.246721,-1.269897,0,0.1
2000-01-01 15:00:00,8.023950,7.629423,0.000186,1020.875625,-10.675546,-1.058556,-0.921751,24.508623,-41.214752,5.0,...,0.295595,-0.000188,-1.572656,-0.214408,0.146392,5.720864,0.139708,-1.191833,0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-13 21:00:00,13.976068,9.267782,0.000962,1013.787969,-5.725973,1.024751,8.022283,23.499479,-48.751160,51.0,...,0.573225,0.001772,1.036875,0.239570,-0.696071,-1.964289,-0.036551,-0.382538,0,0.9
2014-12-14 00:00:00,13.713007,8.729606,-0.000825,1014.931094,-5.089421,3.418641,5.292976,23.499500,-49.022491,52.0,...,-0.538176,-0.001787,1.143125,0.636552,2.393891,-2.729308,0.000021,-0.271332,0,0.9
2014-12-14 03:00:00,14.085962,9.510149,0.000713,1014.133594,-3.518239,3.075252,4.205419,23.383141,-49.258881,53.0,...,0.780543,0.001538,-0.797500,1.571182,-0.343390,-1.087557,-0.116360,-0.236389,0,1.0


In [60]:
data_Y = df["Extreme"]
data_X = df.drop(["Extreme", "LH", "dLH"], axis=1)

desc_corr = abs(df.corr('kendall'))[['LH']].sort_values(by='LH', ascending=False).drop(["LH", "dLH", "Extreme"])

combs = []
for x in range(1, 5):
    combs += [list(y) for y in combinations(desc_corr.index[1:], x)]

res_comb = []
for index in trange(len(combs)):
    X_train, X_test, y_train, y_test = train_test_split(data_X[combs[index]], data_Y, test_size=0.25, random_state=738)
    clf_rbf = svm.SVC(kernel = 'rbf', C = 10)
    clf_rbf.fit(X_train, y_train)
    y_pred = clf_rbf.predict(X_test)
    res_comb.append(f1_score(y_test, y_pred))

  0%|          | 0/4047 [00:00<?, ?it/s]